# import library 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive"

/content/drive/My Drive


In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 25kB/s 
     |████████████████████████████████| 6.6MB 49kB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101
     |████████████████████████████████| 245kB 6.2MB/s 
     |████████████████████████████████| 194kB 9.3MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.1.2-cp36-cp36m-linux_x86_64.whl size=17049202 sha256=85ac687351824e8f5e8df1150b8ec9cca2a18581e3da5edd904d45093b4d7583
  Stored in directory: /root/.cache/pip/wheels/d8/1b/e5/a291cfee98447e0126133246510d01ac2ba5225aac9c2eac2e
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote:

In [ ]:
from mmdet.apis import inference_detector
import mmcv
import glob
import shutil
import os
import torch
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from keras.preprocessing import image
import numpy as np
import cv2

In [ ]:
# load 2 model


mmdetection_path = '/content/drive/My Drive/logo detection: phase 2/model_1.pth'
resnet_path= '/content/drive/My Drive/logo detection: phase 2/model_resnet_dataset8.h5'
mmdetection_model = torch.load(mmdetection_path)
resnet_model = tf.keras.models.load_model(resnet_path)

In [ ]:
# tạo tên class

class_list =[
   'Apple', 'BMW','Heineken','HP','Intel','Mini','Starbucks','Vodafone', 'Unknown', 'Ferrari'
]



#pipe line

In [ ]:
# dùng mmdetection để lấy vị trí logo trong hình và crop ra logo

def detect_logo_location(image_path):
  list_img =[]
  
  img = mmcv.imread(image_path)
  j=0
  dst_dir = "/content/drive/My Drive/logo detection: phase 2/dataset for testing/crop"
  result = inference_detector(mmdetection_model, img) # lấy bouding box từ hình sử dụng model mmdetection
  if result: # nếu tập không rỗng( có tồn tại logo)
    img = Image.open(image_path)
    for i in range(len(result[0])): # loop qua từng bounding box trong hình
      if result[0][i][4] >= 0.3: # nếu probality > 0.5 lấy bouding và crop hình
        left = result[0][i][0]
        top = result[0][i][1]
        right = result[0][i][2]
        bottom = result[0][i][3]
        im_crop = img.crop((left, top, right, bottom))
        if im_crop.mode == "RGBA":
          im_crop = im_crop.convert('RGB')
        name = os.path.join(dst_dir, "crop_" + str(j) + ".jpg")
        im_crop.save(name)
        j=j+1
        list_img.append(name)
  return list_img

In [ ]:
# kiểm tra chất lượng logo

def check_image_quality(list_img):
  list_img2 =[]
  if list_img:
    list_img2 =[]
    for i in range(len(list_img)):
      image_path = list_img[i]
      im = cv2.imread(image_path)
      h, w, c = im.shape
      if h < 32 or w < 32:
        print("logo quá nhỏ")
      else:
        list_img2.append(list_img[i])
  return list_img2

In [ ]:
# dủng resnet để lấy ta xác suất từng class, class cao nhất lấy về 1

def detect_brand_probality(list_img):
  classes_probality = []
  for i in range(len(list_img)):
    image_path = list_img[i]
    img_width, img_height =32,32
    img = image.load_img(image_path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = resnet_model.predict(images)
    classes_probality.append(list(classes[0]))
  return classes_probality

In [ ]:
# trả về tên nhãn hiệu từ xác suất

def class_name(classes_probality):
  Class_name = []
  for i in range(len(classes_probality)):
    list_class = classes_probality[i]
    location = list_class.index(max(list_class))
    Class_name.append(class_list[location])
  return Class_name

In [ ]:
src_dir = "/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
  print(jpgfile)
  image_path = jpgfile
  list_img = detect_logo_location(image_path)
  list_img2 = check_image_quality(list_img)
  if list_img2:
    classes_probality = detect_brand_probality(list_img2)
    print(class_name(classes_probality))
  else:
    print("logo không tồn tại")

/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_8.jpg
['Intel', 'Unknown']
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_5.jpg
['Intel', 'Intel', 'Intel']
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_6.jpg
['Intel', 'Unknown']
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_7.jpg
logo quá nhỏ
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_2.jpg
['Intel', 'Unknown']
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_10.jpg
['Intel', 'Unknown']
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_3.jpg
['Unknown']
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_4.jpg
['Intel', 'Unknown', 'Unknown', 'Unknown']
/content/drive/My Drive/logo detection: phase 2/dataset for testing/intel/intel_1.jpg
['Intel', 'Intel', 'Unknown', 'Intel']
/conte